# Serve an image classification model built in Deeplearning4j with Konduit Serving

This notebook illustrates a simple client-server interaction to perform inference on a Deeplearning4j image classification model using the Python SDK for Konduit Serving.


In [1]:
from konduit import ModelConfig, TensorDataTypesConfig, ModelConfigType, ModelPipelineStep, ParallelInferenceConfig, \
ServingConfig, InferenceConfiguration

from konduit.server import Server
from konduit.client import Client
import numpy as np 

import time
import os 

## Saving models in Deeplearning4j 
The following is a short Java program that loads a simple CNN model from Deeplearning4j's model zoo, initializes weights, then saves the model to a new file, "SimpleCNN.zip". 


```java
import org.deeplearning4j.nn.conf.MultiLayerConfiguration;
import org.deeplearning4j.nn.multilayer.MultiLayerNetwork;
import org.deeplearning4j.zoo.ZooModel;
import org.deeplearning4j.zoo.model.SimpleCNN;

import java.io.File;

public class SaveSimpleCNN {
    private static int nClasses = 5;
    private static boolean saveUpdater = false;

    public static void main(String[] args) throws Exception {
        ZooModel zooModel = SimpleCNN.builder()
            .numClasses(nClasses)
            .inputShape(new int[]{3, 224, 224})
            .build();
        MultiLayerConfiguration conf = ((SimpleCNN) zooModel).conf();
        MultiLayerNetwork net = new MultiLayerNetwork(conf);
        net.init();
        System.out.println(net.summary());
        File locationToSave = new File("SimpleCNN.zip");
        net.save(locationToSave, saveUpdater);
    }
}

```

A reference Java project using Deeplearning4j 1.0.0-beta5 is provided in this repository with a Maven `pom.xml` dependencies file. If using the IntelliJ IDEA IDE, open the `java` folder as a Maven project and run the `main` function of the `SaveSimpleCNN` class. 

## Configuring the server 

In the code, we recognised that SimpleCNN is configured as a MultiLayerNetwork, in contrast with the ComputationGraph class, which is used for more complex networks. Pass this to the `model_type` argument in `ModelConfigType`. 

Note that Konduit Serving configurations require `input_names` and `output_names`. To find the names of input and output nodes in Deeplearning4j, 
- for `input_names`, print the first element of `net.getLayerNames()`.
- for `output_names`, check the last layer when printing `net.summary()`. 

In [2]:
input_data_types = {"image_array": "FLOAT"}
input_names = list(input_data_types.keys())
output_names = ["output"]
port = np.random.randint(1000, 65535)

dl4j_config = ModelConfig(
    tensor_data_types_config=TensorDataTypesConfig(
        input_data_types=input_data_types
    ), 
    model_config_type=ModelConfigType(
        model_type="MULTI_LAYER_NETWORK", 
        model_loading_path=os.path.abspath("../data/multilayernetwork/SimpleCNN.zip")
    )
)

The remaining configurations are similar to how you would configure a TensorFlow model for serving. 

In [3]:
dl4j_pipeline_step = ModelPipelineStep(
    model_config=dl4j_config,
    parallel_inference_config=ParallelInferenceConfig(workers=1),
    input_names=input_names,
    output_names=output_names
)

In [4]:
serving_config = ServingConfig(
    http_port=port,
    input_data_type='NUMPY',
    output_data_type='NUMPY'
)

In [5]:
server = Server(
    config=InferenceConfiguration(
        serving_config=serving_config,
        pipeline_steps=[dl4j_pipeline_step]
    )
)

## Running the server

We generate a (3, 224, 224) array of random numbers between 0 and 255 as input to the model for prediction. Before requesting for a prediction, we normalize the image to be between 0 and 1: 

In [6]:
rand_image = np.random.randint(255, size=(3, 224, 224)) / 255
# note - documentation on how to determine the required shape will be helpful. 

In [7]:
client = Client(
    input_names=input_names,
    output_names=output_names,
    input_type='NUMPY',
    endpoint_output_type='NUMPY',
    return_output_type="NUMPY",
    url='http://localhost:' + str(port)
)

In [8]:
server.start()
time.sleep(30)

prediction = client.predict({"image_array": rand_image})
print(prediction)

server.stop()

Wrote config.json to path C:\Users\Skymind AI Berhad\Documents\konduit-serving-examples\python\config.json
Running with args
java -cp konduit.jar ai.konduit.serving.configprovider.KonduitServingMain --configPath C:\Users\Skymind AI Berhad\Documents\konduit-serving-examples\python\config.json --verticleClassName ai.konduit.serving.verticles.inference.InferenceVerticle


C:\Users\Skymind AI Berhad\AppData\Local\Continuum\miniconda3\lib\site-packages\konduit-0.1.1-py3.7.egg\konduit\client.py:151: FutureWarning: Possible nested set at position 3


[[2.7756006e-02 3.7494433e-01 2.2917205e-02 3.7529669e-05 5.7434499e-01]]
